In [201]:
import gymnasium as gym
import numpy as np
from gymnasium import spaces

In [202]:
env=gym.make('FrozenLake-v1', map_name="4x4", is_slippery=True)

In [203]:
"""
env.P[state][action] probability,nextstate,reward,isterminal

16 states, 4 actions

"""

num_states=env.observation_space.n
num_actions=env.action_space.n

In [204]:
env.P

c:\Python\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.P to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.P` for environment variables or `env.get_wrapper_attr('P')` that will search the reminding wrappers.
  logger.warn(


{0: {0: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  2: [(0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)],
  3: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 1: {0: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False)],
  2: [(0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  3: [(0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 2:

Value Iteration Algorithm

In [205]:

def Valueiteration(env, value_vector, gamma, max_iters, epsilon):
    num_iters = 0
    value_curr = value_vector.copy()
    value_next = value_vector.copy()
    
    for i in range(max_iters):
        num_iters += 1
        for state in env.P:
            max_action = 0 
            for action in env.P[state]:
                temp = 0
                for p, next_state, reward, isTerminal in env.P[state][action]:
                    temp += p * (reward + gamma * value_curr[next_state])
                max_action = max(max_action, temp)
            value_next[state] = max_action

        if np.max(np.abs(value_next - value_curr)) < epsilon:
            value_curr = value_next.copy()
            break
        value_curr = value_next.copy()
    print(num_iters)
    return value_curr




In [206]:
gamma=0.8
max_iters=1000
epsilon=1e-4
value_vector=np.zeros(num_states)
Valueiteration(env,value_vector,gamma,max_iters,epsilon)

23


array([0.01515046, 0.01535729, 0.02728922, 0.01550279, 0.02655761,
       0.        , 0.05967895, 0.        , 0.05814652, 0.13358585,
       0.19659954, 0.        , 0.        , 0.24638195, 0.54410951,
       0.        ])

Stochastic policy Evaluation and Improvement

In [207]:
def follow_stochastic_policy(env,value_vector,policy,gamma,max_iters,epsilon):
    num_states=env.observation_space.n
    value_curr=value_vector
    for i in range(max_iters):
        value_next=value_curr.copy()
        for state in env.P:
            outsum=0
            for action in env.P[state]:
                insum=0
                for p,next_state,reward,isTerminal in env.P[state][action]:
                    insum+=(p*((gamma*value_curr[next_state])+reward))
                outsum+=insum*policy[state][action]
            value_next[state]=outsum
        if np.linalg.norm(value_next-value_curr)<epsilon:
            value_curr=value_next.copy()
            break
        value_curr=value_next.copy()
    return value_curr
    

In [208]:
def improve_stochastic_policy(env,value_vector,gamma):
    num_states=env.observation_space.n
    num_actions=env.action_space.n

    qvaluesMatrix=np.zeros((num_states,num_actions))

    improvedPolicy=np.zeros((num_states,num_actions))
     
    for state in range(num_states):

        for action in range(num_actions):
  
            for p,next_state,reward,isTerminal in env.P[state][action]:
                qvaluesMatrix[state,action]+=(p*(reward+gamma*value_vector[next_state]))
             
  
        bestaction=np.where(qvaluesMatrix[state,:]==np.max(qvaluesMatrix[state,:]))
     
        improvedPolicy[state,bestaction]=1/np.size(bestaction)
    return improvedPolicy,qvaluesMatrix

In [209]:

policy=np.ones((num_states,num_actions))/num_actions
gamma=0.8
max_iters=1000
epsilon=1e-5
value_vector=np.zeros(16)

num_iters=0

for iteration in range(1000):
    num_iters+=1
    if (iteration == 0):
        currentPolicy=policy
    value_vector=follow_stochastic_policy(env,value_vector,currentPolicy,gamma,max_iters,epsilon)
    improvedPolicy,qvaluesMatrix=improve_stochastic_policy(env,value_vector,gamma)
    if np.allclose(currentPolicy,improvedPolicy):
        currentPolicy=improvedPolicy
        print("converged!")
        break
    
    currentPolicy=improvedPolicy



    
    

converged!


In [210]:
print(num_iters)
print(value_vector)
currentPolicy

2
[0.01542077 0.01557963 0.0274332  0.01567188 0.02683969 0.
 0.05977567 0.         0.05840101 0.13377427 0.19672968 0.
 0.         0.24653087 0.54419182 0.        ]


array([[0.  , 0.5 , 0.5 , 0.  ],
       [0.  , 0.  , 0.  , 1.  ],
       [0.  , 0.  , 1.  , 0.  ],
       [0.  , 0.  , 0.  , 1.  ],
       [1.  , 0.  , 0.  , 0.  ],
       [0.25, 0.25, 0.25, 0.25],
       [0.5 , 0.  , 0.5 , 0.  ],
       [0.25, 0.25, 0.25, 0.25],
       [0.  , 0.  , 0.  , 1.  ],
       [0.  , 1.  , 0.  , 0.  ],
       [1.  , 0.  , 0.  , 0.  ],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.  , 0.  , 1.  , 0.  ],
       [0.  , 1.  , 0.  , 0.  ],
       [0.25, 0.25, 0.25, 0.25]])

Policy Iteration Algorithm

In [211]:
def follow_deterministic_policy(env,value_vector,policy,gamma,max_iters,epsilon):
    num_states=env.observation_space.n
    value_curr=value_vector
    for i in range(max_iters):
        value_next=value_curr.copy()
        for state in env.P:
            insum=0
            for p,next_state,reward,isTerminal in env.P[state][policy[state]]:
                insum+=(p*((gamma*value_curr[next_state])+reward))
            value_next[state]=insum
        if np.linalg.norm(value_next-value_curr)<epsilon:
            value_curr=value_next.copy()
            break
        value_curr=value_next.copy()
    return value_curr

In [212]:
def improve_deterministic_policy(env,value_vector,gamma):

    num_states=env.observation_space.n
    num_actions=env.action_space.n

    improved_policy=np.zeros(num_states)
    for state in range(num_states):
        max_action=0
        actions=[]
        for action in range(num_actions):
            action_sum=0
            for p,next_state,reward,isTerminal in env.P[state][action]:
                action_sum+=(p*(reward+gamma*value_vector[next_state]))
            actions.append(action_sum)
        max_action=actions.index(max(actions))
        improved_policy[state]=max_action
    return improved_policy

In [213]:
policy=np.zeros(num_states)
gamma=0.8
max_iters=1000
epsilon=1e-5
value_vector=np.zeros(num_states)

num_iters=0

for iteration in range(max_iters):
    num_iters+=1
    if (iteration == 0):
        currentPolicy=policy
    value_vector=follow_deterministic_policy(env,value_vector,currentPolicy,gamma,max_iters,epsilon)
    improvedPolicy=improve_deterministic_policy(env,value_vector,gamma)
    if np.allclose(currentPolicy,improvedPolicy):
        currentPolicy=improvedPolicy
        print("converged!")
        break
    
    currentPolicy=improvedPolicy

converged!


In [214]:
print(num_iters)
print(value_vector)
currentPolicy

6
[0.01542235 0.01558301 0.02743906 0.01567911 0.02684107 0.
 0.05977898 0.         0.05840331 0.13377752 0.19673287 0.
 0.         0.24653411 0.54419385 0.        ]


array([1., 3., 2., 3., 0., 0., 0., 0., 3., 1., 0., 0., 0., 2., 1., 0.])

2 (d) - Custom Environment 

In [215]:
class Gridworld:
    def __init__(self,desc=['RRRRR','RWRRR','RWFWG','SRRRR','CCCCC'],eta=0.2):
        """ 
        0:Left
        1:Down
        2:Right
        3:Up
        """
        self.action_space=spaces.Discrete(4)
        """ 
        15:Starting state
        14:Goal State
        """
        self.observation_space=spaces.Discrete(25)
        self.agent_loc=15
        self.goal_state=14
        self.P=None
        """ 
        S:Start State
        W:Wall
        R:Road
        C:Cliff
        F:Goal State 1(reward 1)
        G:Goal State 2
        """
        self.desc=desc
        self.eta=eta

    """ 
    Given s gives index in gridworld
    """
    def state_to_index(self,s):
        row=s //5
        column=s- 5*row
        return[row,column]
    
    """ 
    Gives reward for state
    """
    def reward_state(self,s):
        index=self.state_to_index(s)
        if self.desc[index[0]][index[1]]=='W':
            return 0.0
        if self.desc[index[0]][index[1]]=='R' or self.desc[index[0]][index[1]]=='S':
            return 0.0
        if self.desc[index[0]][index[1]]=='C':
            return -10.0
        if self.desc[index[0]][index[1]]=='G':
            return 10.0
        if self.desc[index[0]][index[1]]=='F':
            return 1.0
    
    """ 
    Returns if a state is terminal or not
    """
    def terminal(self,s):
        index=self.state_to_index(s)
        if self.desc[index[0]][index[1]]=='W' or self.desc[index[0]][index[1]]=='C' or self.desc[index[0]][index[1]]=='F' or self.desc[index[0]][index[1]]=='G':
                return True
        return False


    """ 
    Returns next possible states from s
    """
    def next_states(self,s):

        next_states=[s-1,s+1,s-5,s+5]
        if s%5==0:
            next_states.pop(next_states.index(s-1))
        if s<5:
            next_states.pop(next_states.index(s-5))
        if s>=20:
            next_states.pop(next_states.index(s+5))
        if (s+1)%5==0:
            next_states.pop(next_states.index(s+1)) 
        return next_states
            
    def action_state(self,s,a):
        next_state=None
        if a==0:
            if s%5!=0:
                next_state=s-1
            else:
                next_state=s
        elif a==1:
            if s<20:
                next_state=s+5
            else:
                next_state=s
        elif a==2:
            if (s+1)%5==0:
                next_state=s
            else:
                next_state=s+1
        elif a==3:
            if s<5:
                next_state=s
            else:
                next_state=s-5
        return next_state

    """ 
    returns env.P
    """
    def set_P(self):
        p={}
        for s in range(self.observation_space.n):
            index=self.state_to_index(s)
            if self.desc[index[0]][index[1]]=='W' or  self.desc[index[0]][index[1]]=='C' or  self.desc[index[0]][index[1]]=='F' or  self.desc[index[0]][index[1]]=='G':
                dictionary={
                    0:[(1,s,0.0,True)],
                    1:[(1,s,0.0,True)],
                    2:[(1,s,0.0,True)],
                    3:[(1,s,0.0,True)]
                            }
                p[s]=dictionary
                continue
            

            dictionary={}
            for action in range(self.action_space.n):
                q=[]
                next_state=self.action_state(s,action)
                if next_state==s:
                    next_s=self.next_states(s)
                    q.append((1-self.eta,s,self.reward_state(s),self.terminal(s)))
                    num=len(next_s)
                    for j in range(num):
                        q.append((self.eta/num,next_s[j],self.reward_state(next_s[j]),self.terminal(next_s[j])))
                    dictionary[action]=q
                else:
                    q.append((1-self.eta,next_state,self.reward_state(next_state),self.terminal(next_state)))
                    next_s=self.next_states(s)
                    next_s.pop(next_s.index(next_state))
                    num=len(next_s)
                    for j in range(num):
                        q.append((self.eta/num,next_s[j],self.reward_state(next_s[j]),self.terminal(next_s[j])))
                    dictionary[action]=q
            p[s]=dictionary
        self.P=p
                    

                    

        

In [216]:
""" 
0:Left
1:Down
2:Right
3:Up
"""
desc=['RRRRR','RWRRR','RWFWG','SRRRR','CCCCC']
q=Gridworld(desc=desc,eta=0)

q.set_P()
q.P

{0: {0: [(1, 0, 0.0, False), (0.0, 1, 0.0, False), (0.0, 5, 0.0, False)],
  1: [(1, 5, 0.0, False), (0.0, 1, 0.0, False)],
  2: [(1, 1, 0.0, False), (0.0, 5, 0.0, False)],
  3: [(1, 0, 0.0, False), (0.0, 1, 0.0, False), (0.0, 5, 0.0, False)]},
 1: {0: [(1, 0, 0.0, False), (0.0, 2, 0.0, False), (0.0, 6, 0.0, True)],
  1: [(1, 6, 0.0, True), (0.0, 0, 0.0, False), (0.0, 2, 0.0, False)],
  2: [(1, 2, 0.0, False), (0.0, 0, 0.0, False), (0.0, 6, 0.0, True)],
  3: [(1, 1, 0.0, False),
   (0.0, 0, 0.0, False),
   (0.0, 2, 0.0, False),
   (0.0, 6, 0.0, True)]},
 2: {0: [(1, 1, 0.0, False), (0.0, 3, 0.0, False), (0.0, 7, 0.0, False)],
  1: [(1, 7, 0.0, False), (0.0, 1, 0.0, False), (0.0, 3, 0.0, False)],
  2: [(1, 3, 0.0, False), (0.0, 1, 0.0, False), (0.0, 7, 0.0, False)],
  3: [(1, 2, 0.0, False),
   (0.0, 1, 0.0, False),
   (0.0, 3, 0.0, False),
   (0.0, 7, 0.0, False)]},
 3: {0: [(1, 2, 0.0, False), (0.0, 4, 0.0, False), (0.0, 8, 0.0, False)],
  1: [(1, 8, 0.0, False), (0.0, 2, 0.0, False), 

In [217]:
g=[0.2,0.45,0.9]
for gam in g:
    policy=np.zeros(q.observation_space.n)
    gamma=gam
    max_iters=1000
    epsilon=1e-5
    value_vector=np.zeros(q.observation_space.n)

    num_iters=0

    for iteration in range(max_iters):
        num_iters+=1
        if (iteration == 0):
            currentPolicy=policy
        value_vector=follow_deterministic_policy(q,value_vector,currentPolicy,gamma,max_iters,epsilon)
        improvedPolicy=improve_deterministic_policy(q,value_vector,gamma)
        if np.allclose(currentPolicy,improvedPolicy):
            currentPolicy=improvedPolicy
            print("converged!")
            break
        
        currentPolicy=improvedPolicy
    print(f'number of iterations: {num_iters}\n')
    print(f"optimal policy for gamma={gam}, eta={q.eta} is :\n{currentPolicy}\n")

converged!
number of iterations: 6

optimal policy for gamma=0.2, eta=0 is :
[2. 2. 1. 1. 1. 1. 0. 1. 2. 1. 1. 0. 0. 0. 0. 2. 2. 3. 2. 3. 0. 0. 0. 0.
 0.]

converged!
number of iterations: 6

optimal policy for gamma=0.45, eta=0 is :
[2. 2. 1. 1. 1. 1. 0. 2. 2. 1. 1. 0. 0. 0. 0. 2. 2. 2. 2. 3. 0. 0. 0. 0.
 0.]

converged!
number of iterations: 6

optimal policy for gamma=0.9, eta=0 is :
[2. 2. 1. 1. 1. 1. 0. 2. 2. 1. 1. 0. 0. 0. 0. 2. 2. 2. 2. 3. 0. 0. 0. 0.
 0.]



In [218]:
etas=[0,0.2,0.9]
for et in etas:
    q=Gridworld(desc=desc,eta=et)
    q.set_P()
    policy=np.zeros(q.observation_space.n)
    gamma=0.9
    max_iters=1000
    epsilon=1e-5
    value_vector=np.zeros(q.observation_space.n)

    num_iters=0

    for iteration in range(max_iters):
        num_iters+=1
        if (iteration == 0):
            currentPolicy=policy
        value_vector=follow_deterministic_policy(q,value_vector,currentPolicy,gamma,max_iters,epsilon)
        improvedPolicy=improve_deterministic_policy(q,value_vector,gamma)
        if np.allclose(currentPolicy,improvedPolicy):
            currentPolicy=improvedPolicy
            print("converged!")
            break
        
        currentPolicy=improvedPolicy
    print(f'number of iterations: {num_iters}\n')
    print(f"optimal policy for gamma={gam}, eta={q.eta} is :\n{currentPolicy}\n")

converged!
number of iterations: 6

optimal policy for gamma=0.9, eta=0 is :
[2. 2. 1. 1. 1. 1. 0. 2. 2. 1. 1. 0. 0. 0. 0. 2. 2. 2. 2. 3. 0. 0. 0. 0.
 0.]

converged!
number of iterations: 5

optimal policy for gamma=0.9, eta=0.2 is :
[2. 2. 2. 2. 1. 3. 0. 2. 2. 1. 3. 0. 0. 0. 0. 3. 2. 2. 2. 3. 0. 0. 0. 0.
 0.]

converged!
number of iterations: 3

optimal policy for gamma=0.9, eta=0.9 is :
[1. 1. 0. 0. 0. 2. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0.
 0.]

